In [77]:
camera.stop()
robot.stop()

In [138]:
import traitlets
import cv2
import numpy as np
import pyzed.sl as sl
import math
import numpy as np
import sys
import math
import threading
import motors
import time
import ipywidgets.widgets as widgets
from IPython.display import display
from traitlets.config.configurable import HasTraits

LIGHT_YELLOW = np.array([15, 50, 50])
DARK_YELLOW = np.array([30, 220, 220])

SCALE = 1
FWD_MOTOR_SPEED = 1
ALPHA = .9
display_color = widgets.Image(format='jpeg', width='45%') 
display_depth = widgets.Image(format='jpeg', width='45%')  
layout=widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth],layout=layout) #horizontal 
display(sidebyside) #display the widget

robot = motors.MotorsYukon(mecanum=False)



class Camera(HasTraits):
    color_value = traitlets.Any() # monitor the color_value variable
    cx, cy = traitlets.Any(), traitlets.Any()
    last_dir = None
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        # Create a InitParameters object and set configuration parameters
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA #VGA(672*376), HD720(1280*720), HD1080 (1920*1080) or ...
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA  # Use ULTRA depth mode
        init_params.coordinate_units = sl.UNIT.MILLIMETER  # Use meter units (for depth measurements)

        # Open the camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS: #Ensure the camera has opened succesfully
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

         # Create and set RuntimeParameters after opening the camera
        self.runtime = sl.RuntimeParameters()

        #flag to control the thread
        self.thread_runnning_flag = False

        # Get the height and width
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width,self.height,sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C4, sl.MEM.CPU)

        self.viewport_midpoint = 380
        
        
        self.viewport_width = 500
        self.viewport_bottom = 376
        self.last_dir = ''
        self.smoothed_cx = None
        self.window = []
        self.window_size = 5

    def _capture_frames(self): #For data capturing only

        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
           
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map. Depth is aligned on the left image
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)

                frame = self.image.get_data()
                
                
                #Shades lower segment
                # frame[330:,:] = 0
                #Shades the Upper segment
                frame[:200,:] = 0
                #Shades the Left Segment
                frame[:,:(self.viewport_midpoint - (self.viewport_width // 2))] = 0
                #Shades the Right Segment
                frame[:,(self.viewport_midpoint + (self.viewport_width // 2)):] = 0
                frame = cv2.resize(frame, None, fx=SCALE, fy=SCALE, interpolation=cv2.INTER_AREA)
                frame = cv2.addWeighted(frame, 0.5, np.zeros(frame.shape, frame.dtype), 0, 0.4)
                
                hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
                mask = cv2.inRange(hsv, LIGHT_YELLOW, DARK_YELLOW)
                kernel = np.ones((5,5), np.uint8)
                mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

                _, edges = canny_edge_detection(frame)
                yellow_lines = cv2.bitwise_and(mask, edges)
                
                contours, _ = cv2.findContours(yellow_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
                if contours:

                    all_points = np.vstack(contours)
                    xs = all_points[:, 0, 0]
                    ys = all_points[:, 0, 1]
                    
                    top_point = all_points[np.argmin(ys), 0]
                    left_point = all_points[np.argmin(xs), 0]
                    right_point = all_points[np.argmax(xs), 0]

                    pts = np.array([top_point, left_point, right_point, (self.viewport_midpoint, self.viewport_bottom*.25)])
                    point_used = np.mean(pts, axis=0).astype(int)

                    if self.smoothed_cx is None:
                        self.smoothed_cx = point_used[0]
                    else:
                        self.smoothed_cx = int(ALPHA * point_used[0] + (1-ALPHA) * self.smoothed_cx)

                    self.cx = self.smoothed_cx
                    self.cy = point_used[1]

                    cv2.arrowedLine(frame, (self.viewport_midpoint, self.viewport_bottom), (self.cx, self.cy), (255, 0, 255), 2)
                    cv2.circle(frame, top_point, 5, (255, 0, 0), -1)
                    cv2.circle(frame, left_point, 5, (0, 255, 0), -1)
                    cv2.circle(frame, right_point, 5, (0, 0, 255), -1)
                    
                else:
                    self.cx = None

                self.color_value = self.image.get_data()
                display_color.value = bgr8_to_jpeg(frame)
                display_depth.value = bgr8_to_jpeg(yellow_lines)
                
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread
            self.zed.close()

    @traitlets.observe('color_value')
    def handleFrameUpdate(self, change):
        frame = change['new']
        scale = 0.1
        resized_image = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
        display_color.value = bgr8_to_jpeg(resized_image)

    @traitlets.observe('cx')
    def handleMotion(self, change):
        temp = change['new']
        
        if temp is None:
            c = np.bincount(self.window)
            m = np.argmax(c)
            robot.stop()
            print(m, end='\r')
            if m == 1:
                robot.right(1)
                pass
            elif m == 0:
                robot.left(1)
                pass
            return

        else:
            
            difference = (temp - self.viewport_midpoint) / (self.viewport_width / 2)
            difference = max(-1, min(1, difference))
            
            if difference < 0:
                self.window.append(0) #Left
            else:
                self.window.append(1) #Right

            if len(self.window) > self.window_size:
                self.window.pop(0)

            left_speed = (FWD_MOTOR_SPEED + difference) 
            right_speed = (FWD_MOTOR_SPEED - difference)
    
            robot.frontLeft(left_speed)
            robot.backLeft(left_speed)
            robot.frontRight(right_speed)
            robot.backRight(right_speed)
        print(self.window, end='\r')
        
def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

def canny_edge_detection(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(src=gray, ksize=(3,5), sigmaX=0.5)
    edges = cv2.Canny(blurred, 70, 125)
    return blurred, edges

camera = Camera()
camera.start()

[2025-05-14 09:17:14 UTC][ZED][INFO] Logging level INFO
[2025-05-14 09:17:14 UTC][ZED][INFO] Logging level INFO
[2025-05-14 09:17:14 UTC][ZED][INFO] Logging level INFO
[2025-05-14 09:17:15 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
[2025-05-14 09:17:17 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-05-14 09:17:17 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-05-14 09:17:17 UTC][ZED][INFO] [Init]  Video mode: VGA@100
[2025-05-14 09:17:17 UTC][ZED][INFO] [Init]  Serial Number: S/N 33439269
[2025-05-14 09:17:17 UTC][ZED][WARNING] [Init]  Self-calibration failed. Point the camera towards a more textured and brighter area. Avoid objects closer than 1 meter (Error code: 0x01) 
[0, 0, 0, 0, 1]

In [139]:
camera.stop()
robot.stop()